In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import re 
import datetime

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM, Softmax, Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print(f"{tf.__version__=}")
print(f"{np.__version__=}")
print("nvidia-smi")
!nvidia-smi
print("nvcc version")
!nvcc --version
print("nvinfer version")
!dpkg -l | grep nvinfer
print("TensorRT version")
!dpkg -l | grep TensorRT

# Load & Prepare Data

In [ ]:
GESTRUE=["STATIC", "SLIDE_UP", "SLIDE_DOWN", "SLIDE_LEFT", "SLIDE_RIGHT", "ZOOM_IN", "ZOOM_OUT", "HIGHLIGHT", "ON_YES", "OFF_NO", "END",]
NUM_CLASSES = len(GESTRUE)

In [ ]:
# data_dir = "/content/drive/MyDrive/Colab Notebooks/rosbag/labeled_data/labeled_data_30hz"
data_dir = "/home/ubuntu/FYP-ROS/rosbag/data"
data_files = sorted(os.listdir(f"{data_dir}/data"))
label_files = sorted(os.listdir(f"{data_dir}/label"))
assert len(data_files) == len(label_files), "error; some files are missing"

X = []
y = []

for data_file, label_file in zip(data_files, label_files):
    data_match  = re.search(r"rosbag2_([\w]+-[\w]+)_data.csv", data_file) 
    label_match = re.search(r"rosbag2_([\w]+-[\w]+)_label.csv", label_file) 
    assert data_match.group(1) == label_match.group(1), "error; data and label file names do not match"

    data_path = os.path.join(f"{data_dir}/data", data_file)
    if os.path.isfile(data_path):
        x_raw = pd.read_csv(data_path)
        x_raw["timestamp"] -= x_raw["timestamp"][0]
        x_raw=x_raw.drop(["imu0_to_imu1_rotation_x", "imu0_to_imu1_rotation_y", "imu0_to_imu1_rotation_z", "imu0_to_imu1_rotation_w",
                          "imu0_to_imu2_rotation_x", "imu0_to_imu2_rotation_y", "imu0_to_imu2_rotation_z", "imu0_to_imu2_rotation_w"], axis=1)

        x_raw = x_raw.to_numpy()
        x_raw = np.pad(x_raw, ((0, 500 - x_raw.shape[0]), (0, 0)), 'constant')
        X.append(x_raw)

    label_path = os.path.join(f"{data_dir}/label", label_file)
    if os.path.isfile(label_path):
        y.append(pd.read_csv(label_path)['label'])

X = np.array(X)
y = tf.keras.utils.to_categorical(y, num_classes=NUM_CLASSES)

# print dimension
print(f"{X.shape=}")
print(f"{y.shape=}")

print(f"{X[1]=}")
print(f"{y[100]=}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build Model

In [ ]:
model=Sequential()

model.add(Bidirectional(LSTM(units=64, return_sequences=True), input_shape=(500, 37), name='BiLSTM1'))
model.add(Dropout(0.2, name='Dropout1'))
model.add(Bidirectional(LSTM(units=64, return_sequences=True, name='BiLSTM2')))
model.add(Dropout(0.2, name='Dropout2'))
model.add(Bidirectional(LSTM(units=64), name='BiLSTM3'))
model.add(Dropout(0.2, name='Dropout3'))
model.add(Dense(64, activation='relu', name='Dense1'))
model.add(Dense(NUM_CLASSES, activation='softmax', name='Dense2'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

In [ ]:
# model.load_weights("/home/ubuntu/FYP-ROS/weights/model_lstm_weights-2023_2_24-14_51-acc0.96.h5")

# Training

In [ ]:
early_stopping_monitor = EarlyStopping(patience=4)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=16, callbacks=[early_stopping_monitor])

In [ ]:
print(history.history.keys())
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Evaluate

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"{scores=}")

In [ ]:
y_pred = model.predict(X_test)

print(f"{np.argmax(y_pred, axis=1)=}")
print(f"{np.argmax(y_test, axis=1)=}")

error_data_index = []
for i, (yp, yt) in enumerate(zip(np.argmax(y_pred, axis=1), np.argmax(y_test, axis=1))):
    if yp != yt:
        print(f"index: {i}, predicted: {GESTRUE[yp]}, actual: {GESTRUE[yt]}")
        error_data_index.append(i)

In [ ]:
y_test_not_onehot = np.argmax(y_test, axis=1)
y_pred_not_onehot = np.argmax(y_pred, axis=1)
matrix_confusion = confusion_matrix(y_pred_not_onehot, y_test_not_onehot)
sns.heatmap(matrix_confusion, square=True, annot=True, cmap='Blues', fmt='d')
plt.xlabel('predictions')
plt.ylabel('ground truth')

In [ ]:
print(f"total error data size: {len(error_data_index)}")
index = error_data_index[4]

true_label = np.argmax(y_test[index])
predicted_class = np.argmax(y_pred[index])

print(f"{GESTRUE[true_label]=}")
print(f"{GESTRUE[predicted_class]=}")

fig, axs = plt.subplots(3, 6, figsize=(20, 10))

raw_data = X_test[index]
acc_axes = axs[:, :3].ravel()
vel_axes = axs[:, 3:].ravel()
acc_data = np.concatenate([raw_data[:,1:4], raw_data[:,11:14], raw_data[:,21:24]], axis=1)
vel_data = np.concatenate([raw_data[:,4:7], raw_data[:,14:17], raw_data[:,24:27]], axis=1)
acc_titles = [f'Imu{i}_acc_{xyz}' for i in range(3) for xyz in ['x', 'y', 'z']]
vel_titles = [f'Imu{i}_vel_{xyz}' for i in range(3) for xyz in ['x', 'y', 'z']]

for ax, data, title in zip(acc_axes, acc_data.T, acc_titles):
    ax.plot(data)
    ax.set_title(title)
    ax.set_ylim([-1.5 * 9.8, 1.5 * 9.8])
    
for ax, data, title in zip(vel_axes, vel_data.T, vel_titles):
    ax.plot(data)
    ax.set_title(title)
    ax.set_ylim([-5, 5])

fig.suptitle(f'labeled: {GESTRUE[true_label]}, predicted: {GESTRUE[predicted_class]}')
plt.plot()

# Save model

In [ ]:
t = datetime.datetime.now()
t_str = f"{t.year}_{t.month}_{t.day}-{t.hour}_{t.minute}"
acc_str = f"{scores[1]:.2f}" 
model.save_weights(f'/home/ubuntu/FYP-ROS/weights/model_lstm_weights-{t_str}-acc{acc_str}.h5')

# Explaination 

In [ ]:
# !pip install shap
# import shap

In [ ]:
# explainer = shap.Explainer(model)
# shap_values = explainer(X_train.transpose(0, 2, 1))
# shap.summary_plot(shap_values, X_train.transpose(0, 2, 1))